In [1]:
import os
print(os.getcwd())

c:\Users\samsung\Downloads\SP500_Pridiction\버전1(top60)\data\code


In [2]:
import pandas as pd

# 파일 불러오기
df = pd.read_csv("../raw_data/투자지표-ver.csv")

# 날짜 형식 변환 (예: 25.09.30 → 2025-09-30)
df['날짜'] = pd.to_datetime(df['날짜'], format='%y.%m.%d')

# 기업별 정렬
df = df.sort_values(['기업', '날짜'])

# 기업별로 날짜 전체 채우기
target_end = pd.Timestamp("2025-11-07")
filled_df = []

for company, g in df.groupby('기업'):
    full_range = pd.date_range(g['날짜'].min(), target_end, freq='D')
    g = g.set_index('날짜').reindex(full_range)
    g['기업'] = company
    g = g.ffill()  # 이전값으로 채우기
    filled_df.append(g)

# 합치기 및 정리
filled_df = pd.concat(filled_df).reset_index().rename(columns={'index': '날짜'})

filled_df.head()

,날짜,기업,회계연도,분기,주당순이익\nEPS(달러),주당순자산\nBPS(달러),주당매출액\nSPS(달러),주당현금흐름\nCPS(달러),주당배당금\nDPS(달러),주가수익배수\nPER(배),주가순자산배수\nPBR(배),주가매출액배수\nPSR(배),주가현금흐름\n배수 PCR(배),자기자본이익률\nROE(%),총자산이익률\nROA(%),투하자본이익률\nROIC(%),부채비율(%),유동비율(%),이자보상\n배수(배)
0,2021-12-25,AAPL,2022.0,FY1Q,6.03,4.39,23.08,6.85,0.87,28.76,40.21,7.65,25.77,149.8,28.7,38.0,429.9,103.8,43.0
1,2021-12-26,AAPL,2022.0,FY1Q,6.03,4.39,23.08,6.85,0.87,28.76,40.21,7.65,25.77,149.8,28.7,38.0,429.9,103.8,43.0
2,2021-12-27,AAPL,2022.0,FY1Q,6.03,4.39,23.08,6.85,0.87,28.76,40.21,7.65,25.77,149.8,28.7,38.0,429.9,103.8,43.0
3,2021-12-28,AAPL,2022.0,FY1Q,6.03,4.39,23.08,6.85,0.87,28.76,40.21,7.65,25.77,149.8,28.7,38.0,429.9,103.8,43.0
4,2021-12-29,AAPL,2022.0,FY1Q,6.03,4.39,23.08,6.85,0.87,28.76,40.21,7.65,25.77,149.8,28.7,38.0,429.9,103.8,43.0


In [5]:
filled_df["날짜"] = pd.to_datetime(filled_df["날짜"], errors="coerce")
filled_df = filled_df.sort_values(["기업", "날짜"]).reset_index(drop=True)

# 2) 지표 컬럼 자동 선정 (날짜/기업/회계연도/분기 제외)
drop_cols = {"날짜", "기업", "회계연도", "분기"}
value_cols = [c for c in df.columns if c not in drop_cols]

# 3) (날짜, 기업) 중복 처리: 같은 키에 여러 행이 있으면 최근(아래) 행 기준
#    -> 지표별로 마지막 유효값만 남기도록 groupby().last()
dedup = (filled_df
         .groupby(["날짜", "기업"], as_index=False)[value_cols]
         .last())

# 4) 피벗: 날짜 인덱스 × (지표 × 기업) → 이후 계층 순서 교체 (기업, 지표)
wide = dedup.pivot(index="날짜", columns="기업", values=value_cols)

# 현재 컬럼 순서는 (지표, 기업)일 가능성이 높음 → (기업, 지표)로 바꾸기
if isinstance(wide.columns, pd.MultiIndex) and wide.columns.nlevels == 2:
    wide = wide.swaplevel(0, 1, axis=1).sort_index(axis=1, level=[0,1])

# 5) 날짜 기준 정렬
wide = wide.sort_index()

# 6) (선택) 일자 누락 보간: 이전값으로 채우기 (기업·지표별 컬럼 단위)
wide = wide.groupby(level=0, axis=1).ffill()  # level=0은 기업
wide.head()

C:\Users\samsung\AppData\Local\Temp\ipykernel_16372\1991377805.py:25: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  wide = wide.groupby(level=0, axis=1).ffill()  # level=0은 기업


기업            AAPL                                                      \
           부채비율(%) 유동비율(%) 이자보상\n배수(배) 자기자본이익률\nROE(%) 주가매출액배수\nPSR(배)   
날짜                                                                       
2021-10-02     NaN     NaN         NaN             NaN             NaN   
2021-10-03     NaN     NaN         NaN             NaN             NaN   
2021-10-04     NaN     NaN         NaN             NaN             NaN   
2021-10-05     NaN     NaN         NaN             NaN             NaN   
2021-10-06     NaN     NaN         NaN             NaN             NaN   

기업                                                                          \
           주가수익배수\nPER(배) 주가순자산배수\nPBR(배) 주가현금흐름\n배수 PCR(배) 주당매출액\nSPS(달러)   
날짜                                                                           
2021-10-02            NaN             NaN               NaN            NaN   
2021-10-03            NaN             NaN               NaN            NaN   
2021-10-04            NaN             NaN               NaN            NaN   
2021-10-05            NaN             NaN               NaN            NaN   
2021-10-06            NaN             NaN               NaN            NaN   

기업                         ...            XOM                  \
           주당배당금\nDPS(달러)  ... 주가수익배수\nPER(배) 주가순자산배수\nPBR(배)   
날짜                         ...                                  
2021-10-02            NaN  ...            NaN             NaN   
2021-10-03            NaN  ...            NaN             NaN   
2021-10-04            NaN  ...            NaN             NaN   
2021-10-05            NaN  ...            NaN             NaN   
2021-10-06            NaN  ...            NaN             NaN   

기업                                                                         \
           주가현금흐름\n배수 PCR(배) 주당매출액\nSPS(달러) 주당배당금\nDPS(달러) 주당순이익\nEPS(달러)   
날짜                                                                          
2021-10-02               NaN            NaN            NaN            NaN   
2021-10-03               NaN            NaN            NaN            NaN   
2021-10-04               NaN            NaN            NaN            NaN   
2021-10-05               NaN            NaN            NaN            NaN   
2021-10-06               NaN            NaN            NaN            NaN   

기업                                                                         
           주당순자산\nBPS(달러) 주당현금흐름\nCPS(달러) 총자산이익률\nROA(%) 투하자본이익률\nROIC(%)  
날짜                                                                         
2021-10-02            NaN             NaN            NaN              NaN  
2021-10-03            NaN             NaN            NaN              NaN  
2021-10-04            NaN             NaN            NaN              NaN  
2021-10-05            NaN             NaN            NaN              NaN  
2021-10-06            NaN             NaN            NaN              NaN  

[5 rows x 885 columns]

In [4]:
import pandas as pd

# wide가 이미 존재한다고 가정

# 컬럼 구조 확인 후 이름 변환
if isinstance(wide.columns, pd.MultiIndex):
    # (기업, 지표) → "기업_지표"
    new_columns = wide.columns.get_level_values(0).astype(str) + "_" + wide.columns.get_level_values(1).astype(str)
else:
    new_columns = wide.columns.astype(str)

wide.columns = new_columns

# 날짜 컬럼이 있으면 인덱스로 지정, 없으면 그대로 유지
if "날짜" in wide.columns:
    wide["날짜"] = pd.to_datetime(wide["날짜"], errors="coerce")
    wide = wide.set_index("날짜").sort_index()

# 확인
print("날짜를 인덱스로 설정 완료")
print(wide.head())
print(wide.index.name)

날짜를 인덱스로 설정 완료
           AAPL_부채비율(%) AAPL_유동비율(%) AAPL_이자보상\n배수(배) AAPL_자기자본이익률\nROE(%)  \
날짜                                                                           
2021-10-02          NaN          NaN              NaN                  NaN   
2021-10-03          NaN          NaN              NaN                  NaN   
2021-10-04          NaN          NaN              NaN                  NaN   
2021-10-05          NaN          NaN              NaN                  NaN   
2021-10-06          NaN          NaN              NaN                  NaN   

           AAPL_주가매출액배수\nPSR(배) AAPL_주가수익배수\nPER(배) AAPL_주가순자산배수\nPBR(배)  \
날짜                                                                         
2021-10-02                  NaN                 NaN                  NaN   
2021-10-03                  NaN                 NaN                  NaN   
2021-10-04                  NaN                 NaN                  NaN   
2021-10-05                  NaN                 NaN       

In [6]:
# wide의 인덱스가 날짜로 설정되어 있다고 가정
wide.index = pd.to_datetime(wide.index, errors="coerce")

# 2022-11-06 이후 데이터만 필터링
filtered = wide[wide.index > "2022-11-06"]



In [7]:
filtered

기업            AAPL                                                      \
           부채비율(%) 유동비율(%) 이자보상\n배수(배) 자기자본이익률\nROE(%) 주가매출액배수\nPSR(배)   
날짜                                                                       
2022-11-07   596.2    87.9           -           160.9            6.13   
2022-11-08   596.2    87.9           -           160.9            6.13   
2022-11-09   596.2    87.9           -           160.9            6.13   
2022-11-10   596.2    87.9           -           160.9            6.13   
2022-11-11   596.2    87.9           -           160.9            6.13   
...            ...     ...         ...             ...             ...   
2025-11-03   387.2    89.3           -           164.0            9.11   
2025-11-04   387.2    89.3           -           164.0            9.11   
2025-11-05   387.2    89.3           -           164.0            9.11   
2025-11-06   387.2    89.3           -           164.0            9.11   
2025-11-07   387.2    89.3           -           164.0            9.11   

기업                                                                          \
           주가수익배수\nPER(배) 주가순자산배수\nPBR(배) 주가현금흐름\n배수 PCR(배) 주당매출액\nSPS(달러)   
날짜                                                                           
2022-11-07          24.22           47.71             19.79          24.32   
2022-11-08          24.22           47.71             19.79          24.32   
2022-11-09          24.22           47.71             19.79          24.32   
2022-11-10          24.22           47.71             19.79          24.32   
2022-11-11          24.22           47.71             19.79          24.32   
...                   ...             ...               ...            ...   
2025-11-03          33.85           51.42             34.01          27.84   
2025-11-04          33.85           51.42             34.01          27.84   
2025-11-05          33.85           51.42             34.01          27.84   
2025-11-06          33.85           51.42             34.01          27.84   
2025-11-07          33.85           51.42             34.01          27.84   

기업                         ...            XOM                  \
           주당배당금\nDPS(달러)  ... 주가수익배수\nPER(배) 주가순자산배수\nPBR(배)   
날짜                         ...                                  
2022-11-07            0.9  ...           7.02            1.96   
2022-11-08            0.9  ...           7.02            1.96   
2022-11-09            0.9  ...           7.02            1.96   
2022-11-10            0.9  ...           7.02            1.96   
2022-11-11            0.9  ...           7.02            1.96   
...                   ...  ...            ...             ...   
2025-11-03           1.02  ...          16.05            1.85   
2025-11-04           1.02  ...          16.05            1.85   
2025-11-05           1.02  ...          16.05            1.85   
2025-11-06           1.02  ...          16.05            1.85   
2025-11-07           1.02  ...          16.05            1.85   

기업                                                                         \
           주가현금흐름\n배수 PCR(배) 주당매출액\nSPS(달러) 주당배당금\nDPS(달러) 주당순이익\nEPS(달러)   
날짜                                                                          
2022-11-07              4.77          96.75           3.52          12.25   
2022-11-08              4.77          96.75           3.52          12.25   
2022-11-09              4.77          96.75           3.52          12.25   
2022-11-10              4.77          96.75           3.52          12.25   
2022-11-11              4.77          96.75           3.52          12.25   
...                      ...            ...            ...            ...   
2025-11-03              9.33          78.19           3.96           6.88   
2025-11-04              9.33          78.19           3.96           6.88   
2025-11-05              9.33          78.19           3.96           6.88   
2025-11-06              9.33       

In [8]:
# 저장
output_path = "..\output_data\투자지표_44.csv"
filtered.to_csv(output_path, encoding="utf-8-sig")

print("2022-11-06 이후 데이터만 저장 완료")
print("저장 경로:", output_path)

2022-11-06 이후 데이터만 저장 완료
저장 경로: ..\output_data\투자지표_44.csv
